In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import Image
sns.set()

In [2]:
pip install matplotlib_venn --upgrade --user

  Created wheel for matplotlib-venn: filename=matplotlib_venn-0.11.6-py3-none-any.whl size=32065 sha256=6ac4a27a63851176edba3fe0a0e6c304e034f89e51037f661e0a2cf425874c08
  Stored in directory: c:\users\fassg\appdata\local\pip\cache\wheels\42\1c\65\1a733895cd94885d1a97eee84ec7595ff7fddc277a45bcacf5
Successfully built matplotlib-venn
Note: you may need to restart the kernel to use updated packages.


https://www.glerl.noaa.gov/metdata/mil/archive/ 

In [3]:
pip install pandas --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numexpr --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install seaborn --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scipy --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [11]:
original_data = pd.read_csv("mkecallswheader.csv")

In [12]:
original_data.head(10)

,id,time,location,district,nature,status,inserted_at,updated_at,point,call_id
0,2093116,2019-05-21 15:19:03,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,2019-05-21 20:51:09,2019-05-21 20:51:09,0101000020E6100000FC7C94111793454061D971683600...,191411633
1,2093127,2019-05-21 15:24:30,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,2019-05-21 20:57:11,2019-05-21 20:57:11,0101000020E6100000D2AB014A4386454067C416CCA9FC...,191411672
2,2093141,2019-05-21 15:25:46,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,2019-05-21 21:00:12,2019-05-21 21:00:12,0101000020E610000053FFC5D8AE8B45402CAE3B270700...,191411674
3,2093805,2019-05-21 20:46:28,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,2019-05-22 02:22:32,2019-05-22 02:22:32,0101000020E610000078ABF8D04F804540633ABE0779FA...,191412545
4,2093816,2019-05-21 20:50:03,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,2019-05-22 02:25:33,2019-05-22 02:25:33,0101000020E6100000E8323509DE834540C3D7D7BAD4FB...,191412465
5,2093829,2019-05-21 21:02:37,"E WRIGHT ST / N WEIL ST,MKE",5,PARK AND WALK,Service in Progress,2019-05-22 02:37:36,2019-05-22 02:37:36,0101000020E6100000DEF1DC312B88454059D878558CF9...,191412584
6,2093872,2019-05-21 20:50:47,"9010 N 95TH ST,MKE",4,WELFARE CITIZEN,Advised,2019-05-22 02:52:43,2019-05-22 02:52:43,0101000020E6100000357D76C07597454080B4FF01D601...,191412544
7,2093887,2019-05-21 21:25:33,"983 W ARTHUR AV,MKE",2,BATTERY DV,Service in Progress,2019-05-22 03:01:48,2019-05-22 03:01:48,0101000020E6100000BC033C69E17F454041ABDDC02EFB...,191412632
8,2093918,2019-05-21 21:36:05,"4115 N 56TH ST,MKE",7,RETURN STATION,Assignment Completed,2019-05-22 03:16:51,2019-05-22 03:16:51,0101000020E610000014483FD0C08B45404EB747CAF1FE...,191412656
9,2093929,2019-05-21 21:45:53,"7806 W HAMPTON AV,MKE",7,TRAFFIC STOP,Advised,2019-05-22 03:22:52,2019-05-22 03:22:52,0101000020E6100000F6F0C05B7B8D45404F34B4A69E00...,191412676


In [13]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id           int64 
 1   time         object
 2   location     object
 3   district     object
 4   nature       object
 5   status       object
 6   inserted_at  object
 7   updated_at   object
 8   point        object
 9   call_id      int64 
dtypes: int64(2), object(8)
memory usage: 307.3+ MB


In [14]:
original_data.describe()

,id,call_id
count,4.027695e+06,4.027695e+06
mean,5.589211e+06,1.891808e+08
std,2.392488e+07,1.459309e+07
min,1.000000e+00,1.630815e+08
25%,1.006924e+06,1.732129e+08
50%,2.013848e+06,1.906707e+08
75%,3.020772e+06,2.016115e+08
max,1.635417e+08,2.201010e+08


From the above calls to the [`.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html), and [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) methods it is possible to see that there are $10$ total features. Of these features there are two features that are formatted as integers and eight features formatted as the default object that pandas imports non-numerical features as. From the object classified features there are a few that can be converted to new types. The time, inserted_at, and updated_at column should be formatted as a date time object. District should be converted to a numerical categorical value. Nature and status should be converted to categorical features. Location should be kept as an object feature. More features should be extracted from the location in order to draw further observations. Both street name and street suffix would be good features to extract. The ID and point features are artifacts of the database the police dataset came from and as such will be dropped.

The first step is to copy the data into a new dataframe so that the original data can be preserved

In [17]:
df = original_data.copy(deep=True)

Now convert the time, inserted_at, and updated_at features into date time objects.

In [18]:
df['time'] = pd.to_datetime(df['time'], infer_datetime_format=True)  
df['inserted_at'] = pd.to_datetime(df['inserted_at'], infer_datetime_format=True)  
df['updated_at'] = pd.to_datetime(df['updated_at'], infer_datetime_format=True)

In [19]:
df.head()

,id,time,location,district,nature,status,inserted_at,updated_at,point,call_id
0,2093116,2019-05-21 15:19:03,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,2019-05-21 20:51:09,2019-05-21 20:51:09,0101000020E6100000FC7C94111793454061D971683600...,191411633
1,2093127,2019-05-21 15:24:30,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,2019-05-21 20:57:11,2019-05-21 20:57:11,0101000020E6100000D2AB014A4386454067C416CCA9FC...,191411672
2,2093141,2019-05-21 15:25:46,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,2019-05-21 21:00:12,2019-05-21 21:00:12,0101000020E610000053FFC5D8AE8B45402CAE3B270700...,191411674
3,2093805,2019-05-21 20:46:28,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,2019-05-22 02:22:32,2019-05-22 02:22:32,0101000020E610000078ABF8D04F804540633ABE0779FA...,191412545
4,2093816,2019-05-21 20:50:03,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,2019-05-22 02:25:33,2019-05-22 02:25:33,0101000020E6100000E8323509DE834540C3D7D7BAD4FB...,191412465


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   time         datetime64[ns]
 2   location     object        
 3   district     object        
 4   nature       object        
 5   status       object        
 6   inserted_at  datetime64[ns]
 7   updated_at   datetime64[ns]
 8   point        object        
 9   call_id      int64         
dtypes: datetime64[ns](3), int64(2), object(5)
memory usage: 307.3+ MB


In [20]:
df.describe()

,id,call_id
count,4.027695e+06,4.027695e+06
mean,5.589211e+06,1.891808e+08
std,2.392488e+07,1.459309e+07
min,1.000000e+00,1.630815e+08
25%,1.006924e+06,1.732129e+08
50%,2.013848e+06,1.906707e+08
75%,3.020772e+06,2.016115e+08
max,1.635417e+08,2.201010e+08


From the above calls to the [`.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html), and [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) methods it is possible to see that the time base features are now properly formatted. The next step is to bin the features by hour. This will be done in order to reduce the granularity of the data as well as matching up this dataset with hourly weather data.

In [ ]:
# TODO

Next the id and point columns will be dropped since they are artifacts of the database the data was stored in. The point column is a hexadecimal postgres geometry point object that matches up to a location. Since it can not be reverse interpreted and will be correlated with location we will be dropping it.

In [28]:
df = df.drop('id', axis=1)
# df = df.drop('point', axis=1) ### do not want to drop this as its a postgres geospatial for approx lat long.
df.head()

,time,location,district,nature,status,inserted_at,updated_at,call_id
0,2019-05-21 15:19:03,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,2019-05-21 20:51:09,2019-05-21 20:51:09,191411633
1,2019-05-21 15:24:30,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,2019-05-21 20:57:11,2019-05-21 20:57:11,191411672
2,2019-05-21 15:25:46,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,2019-05-21 21:00:12,2019-05-21 21:00:12,191411674
3,2019-05-21 20:46:28,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,2019-05-22 02:22:32,2019-05-22 02:22:32,191412545
4,2019-05-21 20:50:03,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,2019-05-22 02:25:33,2019-05-22 02:25:33,191412465


The nature, status, and district features will now be converted into categorical features

In [30]:
df['district'].unique()

array(['4', '3', '7', '2', '5', '6', '1', nan, 'OCOE', 'OUT', 'NTF',
       'SPD', 'CITY', 'ICS3', 'CIB', 'DPR', 'TRU', 'SCD', 'ICS', 'SF',
       'NLA', 'ICS1', 'IFC', 'FI', 'SID', 'D0', 'ICS7', 'ICS5', 'JUNE',
       'DDAC', 'MIRT', 'ICS6', 'ID', 'ADMN', 'TEU', 'MID', 'NID'],
      dtype=object)

In [34]:
df['district'].value_counts()

3       673002
7       650109
5       567958
4       552701
2       545253
6       473525
1       448786
CITY      7080
NTF       6274
SPD       5477
OCOE      3316
OUT       2249
DDAC       399
SCD        300
DPR        277
ICS1       229
CIB        218
FI          83
SF          55
NLA         34
ICS3        32
SID         31
ICS         23
TEU         16
ICS5         9
JUNE         5
ICS6         4
IFC          4
TRU          3
D0           2
MIRT         2
ADMN         2
NID          2
ICS7         1
ID           1
MID          1
Name: district, dtype: int64

Running a value counts on the different districts shows that there are more than the expected number of police districts in the city of Milwaukee present in the data. There should only be districts one through seven. Instead the data contains more districts than expected. These districts will be converted into categorical anyways as their entries will be useful for some observations. The erraneous districts will likely be ignored when drawing district based conclusions as it is not known what the other districts mean. District will be converted to a categorical with 36 different categories.

In [31]:
df['nature'].unique()

array(['ACC PI', 'TRAFFIC STOP', 'SUBJ WANTED', 'SPECIAL ASSIGN',
       'TRBL W/SUBJ', 'PARK AND WALK', 'WELFARE CITIZEN', 'BATTERY DV',
       'RETURN STATION', 'BUSINESS CHECK', 'SUSPICIOUS-OTH', 'REPORTS',
       'THEFT', 'VIOL REST ORDER', 'SUBJ WITH GUN', 'TAVERN CHECK',
       'PATROL', 'FOLLOW UP', 'ACC PDO', 'TRAFFIC HAZARD',
       'FAMILY TROUBLE', 'NON PURSUIT', 'BATTERY', 'INVESTIGATION',
       'PRISONER TRANS', 'BUS INV', 'CALL FOR POLICE', 'THEFT VEHICLE',
       'SUSP PERS/AUTO', 'ENTRY TO AUTO', 'ASSIGNMENT', 'PROPERTY DAMAGE',
       'IND EXPOSURE', 'NOISE NUISANCE', 'PROPERTY PICKUP', 'SOLICITING',
       'STOLEN VEHICLE', 'OUT OF SERVICE', 'COURT DUTY', 'ENTRY',
       'PARKING TROUBLE', 'GRAFFITI', 'TRBL W/JUV', 'SHOTS FIRED',
       'SHOTSPOTTER', 'BUS INVESTIGATIO', 'THREAT', 'SUBJ WITH WEAPON',
       'TRAFFIC LASER', 'HOME VISIT DV', 'COMMUNITY MTNG',
       'CITIZEN CONTACT', 'RECK USE OF WEAP', 'ABAND/STOLEN PRO',
       'ASSIGN-ADMN MPD', 'CONVEY PROPERTY',

In [35]:
df['nature'].value_counts()

TRAFFIC STOP        401644
BUSINESS CHECK      325863
TRBL W/SUBJ         237324
RETURN STATION      209934
FOLLOW UP           150661
                     ...  
ACC PD1                  1
ABAND/LOST               1
2831 N 21ST              1
THREAT TO SCHOOL         1
STOLEN PROP              1
Name: nature, Length: 317, dtype: int64

The natures all look normal. They will be converted into a categorical value which will have 317 categories.

In [32]:
df['status'].unique()

array(['Service in Progress', 'City Citation(s) Issued',
       'Assignment Completed', 'Unable to Locate Complainant', 'Advised',
       'To be Filed', 'Advised/Referral', 'No Prosecution',
       'Open Investigation', 'Cleared by Arrest', 'False Alarm',
       'Filed Driver Exchange Report', 'Patrol Request', 'Referral',
       'Ordered to Appear', 'State Citation(s) Issued',
       'False Alarm (Weather Related)'], dtype=object)

In [37]:
df['status'].value_counts()

Service in Progress              1424568
Assignment Completed             1186663
Advised                           615111
Unable to Locate Complainant      336406
To be Filed                       164199
City Citation(s) Issued           153937
Advised/Referral                   87051
Open Investigation                 40803
No Prosecution                      6475
Cleared by Arrest                   4696
False Alarm                         2889
Filed Driver Exchange Report        2631
Referral                            1291
Patrol Request                       697
State Citation(s) Issued             160
False Alarm (Weather Related)         93
Ordered to Appear                     25
Name: status, dtype: int64

Status looks normal as well and will be converted to a categorical value. It will contain 17 different categories.

In [39]:
df['district'] = df['district'].astype("category")
df['nature'] = df['nature'].astype("category")
df['status'] = df['status'].astype('category')

In [40]:
df['district'].dtype

CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', 'ADMN', 'CIB', 'CITY',
                  'D0', 'DDAC', 'DPR', 'FI', 'ICS', 'ICS1', 'ICS3', 'ICS5',
                  'ICS6', 'ICS7', 'ID', 'IFC', 'JUNE', 'MID', 'MIRT', 'NID',
                  'NLA', 'NTF', 'OCOE', 'OUT', 'SCD', 'SF', 'SID', 'SPD',
                  'TEU', 'TRU'],
, ordered=False)

In [41]:
df['nature'].dtype

CategoricalDtype(categories=['.', '0', '1 BLOCK NORTH OF', '1301', '1359', '1603', '1733',
                  '230 N 37TH ST', '2532', '2831 N 21ST',
                  ...
                  'VIOL REST ORD', 'VIOL REST ORD-DV', 'VIOL REST ORDER',
                  'WATER MAIN BREAK', 'WATER MAIN BRK', 'WEAPON',
                  'WELFARE CHK', 'WELFARE CITIZEN', 'WIRES DOWN', '`'],
, ordered=False)

In [42]:
df['status'].dtype

CategoricalDtype(categories=['Advised', 'Advised/Referral', 'Assignment Completed',
                  'City Citation(s) Issued', 'Cleared by Arrest',
                  'False Alarm', 'False Alarm (Weather Related)',
                  'Filed Driver Exchange Report', 'No Prosecution',
                  'Open Investigation', 'Ordered to Appear', 'Patrol Request',
                  'Referral', 'Service in Progress',
                  'State Citation(s) Issued', 'To be Filed',
                  'Unable to Locate Complainant'],
, ordered=False)

The nature, status, and district features are now formatted as categories.

The last step is to extract features from the location category

In [52]:
def get_street_info(address: str) -> (int, str, str, int):
    """
    This method will take in a string representing an address and will return the information present in that address.
    Some example addresses are as follows:
        3526 HIGH ST
        11150 TRINITY RIVER DR Unit 114
        4533 LOCH HAVEN WAY
    :param address: the string passed in representing the address
    :return: a tuple containing (home number, street name, street type, unit number.)
    :auth: Grant Fass
    :since: 6 December 2021
    """
    street_type_lookup = ["ALY", "ANX", "ARC", "AVE", "BYU", "BCH", "BND", "BLF", "BLFS", "BTM", "BLVD", "BR", "BRG", "BRK", "BRKS", "BG", "BGS", "BYP", "CP", "CYN", "CPE",
                          "CSWY", "CTR", "CTRS", "CIR", "CIRS", "CLF", "CLFS", "CLB", "CMN", "CMNS", "COR", "CORS", "CRSE", "CT", "CTS", "CV", "CVS", "CRK", "CRES", "CRST", 
                          "XING", "XRD", "XRDS", "CURV", "DL", "DM", "DV", "DR", "EST", "ESTS", "EXPY", "EXT", "EXTS", "FALL", "FLS", "FRY", "FLD", "FLDS", "FLT", "FLTS", 
                          "FRD", "FRDS", "FRST", "FRG", "FRGS", "FRK", "FRKS", "FT", "FWY", "GDN", "GDNS", "GTWY", "GLN", "GLNS", "GRN", "GRNS", "GRV", "GRVS", "HBR", "HBRS", 
                          "HVN", "HTS", "HWY", "HL", "HLS", "HOLW", "INLT", "IS", "ISS", "ISLE", "JCT", "JCTS", "KY", "KYS", "KNL", "KNLS", "LK", "LKS", "LAND", "LNDG", "LN",
                          "LGT", "LGTS", "LF", "LCK", "LCKS", "LDG", "LOOP", "MALL", "MNR", "MNRS", "MDW", "MDWS", "MEWS", "ML", "MLS", "MSN", "MTWY", "MT", "MTN", "MTNS", 
                          "NCK", "ORCH", "OVAL", "OPAS", "PARK", "PKWY", "PASS", "PSGE", "PATH", "PIKE", "PNE", "PNES", "PL", "PLN", "PLNS", "PLZ", "PT", "PTS", "PRT", "PRTS", 
                          "PR", "RADL", "RAMP", "RNCH", "RPD", "RPDS", "RST", "RDG", "RDGS", "RIV", "RD", "RDS", "RTE", "ROW", "RUE", "RUN", "SHL", "SHLS", "SHR", "SHRS", 
                          "SKWY", "SPG", "SPGS", "SPUR", "SQ", "SQS", "STA", "STRA", "STRM", "ST", "STS", "SMT", "TER", "TRWY", "TRCE", "TRAK", "TRFY", "TRL", "TRLR", "TUNL",
                          "TPKE", "UPAS", "UN", "UNS", "VLY", "VLYS", "VIA", "VW", "VWS", "VLG", "VLGS", "VL", "VIS", "WALK", "WALL", "WAY", "WAYS", "WL", "WLS", "AV"]
    # this is used primarily for error checking
    unmatched_suffix = ""
    # remove the ,MKE suffix from the location if present
    addr = address.removesuffix(",MKE")
    # split apart the address on spaces
    addr = addr.split()
    # get the last element in the address
    last = addr[-1]
    # separate out the house number assuming it will be the first entry in the split array
    house_number = addr[0]
    # remove the house number fromt the split array
    addr = addr[1:]
    # check if a unit number is present, if so record and remove it, otherwise record 'None'
    unit_number = None
    if 'Unit' in addr :
        unit_number = last
        addr = addr[:-2]
    # check if the street type (the last entry in the split array now) is present in the lookup, if so record it, otherwise record Unknown        
    street_suffix = 'Unknown'
    if last in street_type_lookup:
        street_suffix = last
        addr = addr[:-1]
    elif 'HIGHWAY' in address:
        street_suffix = "HWY"
    else:
        unmatched_suffix = last
    
    # now the remaining fields of the list should be the street name
    street_name = ' '.join(addr)
    return (house_number, street_name, street_suffix, unit_number)

In [53]:
street_data_as_tuple = df["location"].map(get_street_info)
street_data_array = street_data_as_tuple.values.tolist()
data = pd.DataFrame(street_data_array, columns=['House_Number', 'Street_Name', 'Street_Suffix', 'Unit_Number'])
data.head(20)

,House_Number,Street_Name,Street_Suffix,Unit_Number
0,7420,W GOOD HOPE,RD,None
1,1421,N 27TH,ST,None
2,4054,N 71ST,ST,None
3,245,W LINCOLN,AV,None
4,1721,W CANAL,ST,None
5,E,WRIGHT ST / N WEIL,ST,None
6,9010,N 95TH,ST,None
7,983,W ARTHUR,AV,None
8,4115,N 56TH,ST,None
9,7806,W HAMPTON,AV,None


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   House_Number   object
 1   Street_Name    object
 2   Street_Suffix  object
 3   Unit_Number    object
dtypes: object(4)
memory usage: 122.9+ MB


In [57]:
data['Unit_Number'].value_counts()

Series([], Name: Unit_Number, dtype: int64)

https://mpd.digitalpublicworks.com/?start=2019-01-05T00:00:00-06:00&end=2019-01-05T23:59:59.999999-06:00

looking at the street data shows that some locations are corners and have two street locations. The algorithm will need to be changed to report both street names seperatly. Unit Number can also be removed as they do not store units apparently. Also the first term seems to be picking up some directional headers like NSEW as numbers for street address which will need to be fixed.